In [ ]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam

from includes.helpers import get_function_train_test_set

In [ ]:
X_train, X_test, y_train, y_test = get_function_train_test_set()

In [ ]:
# Function to create model for Optuna optimization
def create_model(trial):
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    model = Sequential([
      Input(shape=(X_train.shape[1],)),
      Dense(128, activation='relu'),
      Dropout(0.3),
      Dense(64, activation='relu'),
      Dropout(0.2),
      Dense(32, activation='relu'),
      Dropout(0.1),
      Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the objective function for Optuna
def objective(trial):
    model = create_model(trial)
    MODEL_EPOCHS = trial.suggest_int('epochs', 10, 30)
    MODEL_BATCH = trial.suggest_int('batch_size', 32, 128)
    model.fit(X_train, y_train, epochs=MODEL_EPOCHS, batch_size=MODEL_BATCH, verbose=0, validation_data=(X_test, y_test))
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy  # We want to maximize accuracy

In [ ]:
# Create the Optuna study and optimize hyperparameters
study = optuna.create_study(study_name="FN fuzzdistill hyperparam tuning",direction='maximize')
study.optimize(objective, n_trials=50)  # Number of trials to run

# Print the best hyperparameters and the best accuracy
print(f'Best Hyperparameters: {study.best_params}')
print(f'Best Accuracy: {study.best_value}')